In [1]:
# ============================================
# SMART LOGISTICS DECISION SYSTEM
# Phase 1: Data Engineering
# ============================================

import pandas as pd
import numpy as np

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print("Environment Ready ✅")

Environment Ready ✅


In [2]:
# Load dataset
df = pd.read_csv("E:/Projects/Smart-Logistics-System/data/raw/smart_logistics_dataset.csv")

print("Dataset Loaded Successfully ✅")

Dataset Loaded Successfully ✅


In [3]:
# ============================================
# DATA INSPECTION
# ============================================

# 1. Shape
print("Shape of dataset:", df.shape)

# 2. Columns
print("\nColumns:")
print(df.columns)

# 3. Data Types
print("\nData Types:")
print(df.dtypes)

# 4. Missing Values
print("\nMissing Values:")
print(df.isnull().sum())

# 5. First 5 Rows
print("\nFirst 5 Rows:")
display(df.head())

Shape of dataset: (1000, 17)

Columns:
Index(['Timestamp', 'Asset_ID', 'Latitude', 'Longitude', 'Inventory_Level', 'Shipment_Status', 'Temperature', 'Humidity', 'Precipitation(mm)', 'Traffic_Status', 'Waiting_Time', 'User_Transaction_Amount', 'User_Purchase_Frequency', 'Logistics_Delay_Reason', 'Asset_Utilization', 'Demand_Forecast', 'Logistics_Delay'], dtype='object')

Data Types:
Timestamp                   object
Asset_ID                    object
Latitude                   float64
Longitude                  float64
Inventory_Level              int64
Shipment_Status             object
Temperature                float64
Humidity                   float64
Precipitation(mm)          float64
Traffic_Status              object
Waiting_Time                 int64
User_Transaction_Amount      int64
User_Purchase_Frequency      int64
Logistics_Delay_Reason      object
Asset_Utilization          float64
Demand_Forecast              int64
Logistics_Delay              int64
dtype: object

Missi

,Timestamp,Asset_ID,Latitude,Longitude,Inventory_Level,Shipment_Status,Temperature,Humidity,Precipitation(mm),Traffic_Status,Waiting_Time,User_Transaction_Amount,User_Purchase_Frequency,Logistics_Delay_Reason,Asset_Utilization,Demand_Forecast,Logistics_Delay
0,20-03-2024 00:11,Truck_7,-65.7383,11.2497,390,Delayed,27.0,67.8,10.023475,Detour,38,320,4,NaN,60.1,285,1
1,30-10-2024 07:53,Truck_6,22.2748,-131.7086,491,In Transit,22.5,54.3,21.397599,Heavy,16,439,7,Weather,80.9,174,1
2,29-07-2024 18:42,Truck_10,54.9232,79.5455,190,In Transit,25.2,62.2,14.520535,Detour,34,355,3,NaN,99.2,260,0
3,28-10-2024 00:50,Truck_9,42.3900,-1.4788,330,Delivered,25.4,52.3,2.613761,Heavy,37,227,5,Traffic,97.4,160,1
4,27-09-2024 15:52,Truck_7,-65.8477,47.9468,480,Delayed,20.5,57.2,35.412586,Clear,56,197,6,NaN,71.6,270,1


In [4]:
# ============================================
# TARGET DISTRIBUTION
# ============================================

print("Logistics Delay Distribution:")
print(df["Logistics_Delay"].value_counts())

print("\nPercentage Distribution:")
print(df["Logistics_Delay"].value_counts(normalize=True) * 100)

Logistics Delay Distribution:
Logistics_Delay
1    566
0    434
Name: count, dtype: int64

Percentage Distribution:
Logistics_Delay
1    56.6
0    43.4
Name: proportion, dtype: float64


In [5]:
# ============================================
# REMOVE DATA LEAKAGE COLUMN (logistics delay reason)
# ============================================

df = df.drop(columns=["Logistics_Delay_Reason"])

print("Column removed successfully ✅")
print("Updated shape:", df.shape)


Column removed successfully ✅
Updated shape: (1000, 16)


In [6]:
# ============================================
# CONVERT TIMESTAMP TO DATETIME
# ============================================

df["Timestamp"] = pd.to_datetime(df["Timestamp"], dayfirst=True)

print("Timestamp converted successfully ✅")
print("New dtype:", df["Timestamp"].dtype)

Timestamp converted successfully ✅
New dtype: datetime64[ns]


In [7]:
# ============================================
# EXTRACT TIME-BASED FEATURES
# ============================================

df["hour"] = df["Timestamp"].dt.hour
df["day_of_week"] = df["Timestamp"].dt.dayofweek  # Monday=0, Sunday=6

print("Time features extracted successfully ✅")
print(df[["hour", "day_of_week"]].head())

Time features extracted successfully ✅
   hour  day_of_week
0     0            2
1     7            2
2    18            0
3     0            0
4    15            4


In [8]:
# ============================================
# CREATE PEAK HOUR FEATURE
# ============================================

df["peak_hour"] = df["hour"].apply(
    lambda x: 1 if (8 <= x <= 10) or (17 <= x <= 20) else 0
)

print("Peak hour feature created successfully ✅")
print(df[["hour", "peak_hour"]].head())

Peak hour feature created successfully ✅
   hour  peak_hour
0     0          0
1     7          0
2    18          1
3     0          0
4    15          0


In [9]:
# ============================================
# REMOVE ORIGINAL TIMESTAMP
# ============================================

df = df.drop(columns=["Timestamp"])

print("Timestamp column removed successfully ✅")
print("Updated shape:", df.shape)

Timestamp column removed successfully ✅
Updated shape: (1000, 18)


In [10]:
# ============================================
# CHECK UNIQUE VALUES IN CATEGORICAL COLUMNS
# ============================================

categorical_cols = df.select_dtypes(include=["object"]).columns

for col in categorical_cols:
    print(f"\nColumn: {col}")
    print("Unique values:", df[col].nunique())
    print(df[col].unique())


Column: Asset_ID
Unique values: 10
['Truck_7' 'Truck_6' 'Truck_10' 'Truck_9' 'Truck_4' 'Truck_2' 'Truck_8'
 'Truck_3' 'Truck_5' 'Truck_1']

Column: Shipment_Status
Unique values: 3
['Delayed' 'In Transit' 'Delivered']

Column: Traffic_Status
Unique values: 3
['Detour' 'Heavy' 'Clear']


In [11]:
# ============================================
# REMOVE SHIPMENT STATUS (LEAKAGE)
# ============================================

df = df.drop(columns=["Shipment_Status"])

print("Shipment_Status removed successfully ✅")
print("Updated shape:", df.shape)


Shipment_Status removed successfully ✅
Updated shape: (1000, 17)


In [12]:
# ============================================
# Removing redundant traffic status = clear since clear + detour + heavy = 1 always in pandas hot encoding
# ============================================

df = pd.get_dummies(df, columns=["Traffic_Status"], drop_first=True)

print("Traffic_Status encoded successfully ✅")
print(df.head())

Traffic_Status encoded successfully ✅
   Asset_ID  Latitude  Longitude  Inventory_Level  Temperature  Humidity  Precipitation(mm)  Waiting_Time  User_Transaction_Amount  User_Purchase_Frequency  Asset_Utilization  Demand_Forecast  Logistics_Delay  hour  day_of_week  peak_hour  Traffic_Status_Detour  Traffic_Status_Heavy
0   Truck_7  -65.7383    11.2497              390         27.0      67.8          10.023475            38                      320                        4               60.1              285                1     0            2          0                   True                 False
1   Truck_6   22.2748  -131.7086              491         22.5      54.3          21.397599            16                      439                        7               80.9              174                1     7            2          0                  False                  True
2  Truck_10   54.9232    79.5455              190         25.2      62.2          14.520535            34     

In [13]:
# ============================================
# ONE-HOT ENCODE ASSET_ID
# ============================================

df = pd.get_dummies(df, columns=["Asset_ID"], drop_first=True)

print("Asset_ID encoded successfully ✅")
print("Current shape:", df.shape)

Asset_ID encoded successfully ✅
Current shape: (1000, 26)


In [14]:
# ============================================
# VERIFY DATA TYPES
# ============================================

print(df.dtypes)

print("\nNon-numeric columns:")
print(df.select_dtypes(include=["object"]).columns)

Latitude                   float64
Longitude                  float64
Inventory_Level              int64
Temperature                float64
Humidity                   float64
Precipitation(mm)          float64
Waiting_Time                 int64
User_Transaction_Amount      int64
User_Purchase_Frequency      int64
Asset_Utilization          float64
Demand_Forecast              int64
Logistics_Delay              int64
hour                         int32
day_of_week                  int32
peak_hour                    int64
Traffic_Status_Detour         bool
Traffic_Status_Heavy          bool
Asset_ID_Truck_10             bool
Asset_ID_Truck_2              bool
Asset_ID_Truck_3              bool
Asset_ID_Truck_4              bool
Asset_ID_Truck_5              bool
Asset_ID_Truck_6              bool
Asset_ID_Truck_7              bool
Asset_ID_Truck_8              bool
Asset_ID_Truck_9              bool
dtype: object

Non-numeric columns:
Index([], dtype='object')


In [15]:
# ============================================
# FINAL MISSING VALUE CHECK (1)
# ============================================

print("Total missing values:", df.isnull().sum().sum())


Total missing values: 0


In [16]:
# ============================================
# FINAL CHECK FOR DUPLICATES (2)
# ============================================

print("Duplicate rows:", df.duplicated().sum())

Duplicate rows: 0


In [17]:
# ============================================
# DEFINE FEATURES (X) AND TARGET (y)
# ============================================

X = df.drop(columns=["Logistics_Delay"])
y = df["Logistics_Delay"]

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: (1000, 25)
Target shape: (1000,)


In [18]:
# ============================================
# SAVE CLEAN MODEL DATASET
# ============================================

df.to_csv("E:/Projects/Smart-Logistics-System/data/processed/clean_model_dataset.csv", index=False)

print("Clean dataset saved successfully ✅")

Clean dataset saved successfully ✅


In [19]:
# ============================================
# CONVERT BOOLEAN COLUMNS TO INTEGER
# ============================================

bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)

print("Boolean columns converted to 0/1 integers ✅")

Boolean columns converted to 0/1 integers ✅


In [20]:
print(df.dtypes)

Latitude                   float64
Longitude                  float64
Inventory_Level              int64
Temperature                float64
Humidity                   float64
Precipitation(mm)          float64
Waiting_Time                 int64
User_Transaction_Amount      int64
User_Purchase_Frequency      int64
Asset_Utilization          float64
Demand_Forecast              int64
Logistics_Delay              int64
hour                         int32
day_of_week                  int32
peak_hour                    int64
Traffic_Status_Detour        int64
Traffic_Status_Heavy         int64
Asset_ID_Truck_10            int64
Asset_ID_Truck_2             int64
Asset_ID_Truck_3             int64
Asset_ID_Truck_4             int64
Asset_ID_Truck_5             int64
Asset_ID_Truck_6             int64
Asset_ID_Truck_7             int64
Asset_ID_Truck_8             int64
Asset_ID_Truck_9             int64
dtype: object


In [22]:
df.to_csv("E:/Projects/Smart-Logistics-System/data/processed/clean_model_dataset.csv", index=False)

In [23]:
# ============================================
# LOAD AND INSPECT CLEAN DATASET
# ============================================

clean_df = pd.read_csv("E:/Projects/Smart-Logistics-System/data/processed/clean_model_dataset.csv")

print("Shape:", clean_df.shape)
print("\nColumns:")
print(clean_df.columns)
print("\nPreview:")
display(clean_df.head())

Shape: (1000, 26)

Columns:
Index(['Latitude', 'Longitude', 'Inventory_Level', 'Temperature', 'Humidity', 'Precipitation(mm)', 'Waiting_Time', 'User_Transaction_Amount', 'User_Purchase_Frequency', 'Asset_Utilization', 'Demand_Forecast', 'Logistics_Delay', 'hour', 'day_of_week', 'peak_hour', 'Traffic_Status_Detour', 'Traffic_Status_Heavy', 'Asset_ID_Truck_10', 'Asset_ID_Truck_2', 'Asset_ID_Truck_3', 'Asset_ID_Truck_4', 'Asset_ID_Truck_5', 'Asset_ID_Truck_6', 'Asset_ID_Truck_7', 'Asset_ID_Truck_8', 'Asset_ID_Truck_9'], dtype='object')

Preview:


,Latitude,Longitude,Inventory_Level,Temperature,Humidity,Precipitation(mm),Waiting_Time,User_Transaction_Amount,User_Purchase_Frequency,Asset_Utilization,Demand_Forecast,Logistics_Delay,hour,day_of_week,peak_hour,Traffic_Status_Detour,Traffic_Status_Heavy,Asset_ID_Truck_10,Asset_ID_Truck_2,Asset_ID_Truck_3,Asset_ID_Truck_4,Asset_ID_Truck_5,Asset_ID_Truck_6,Asset_ID_Truck_7,Asset_ID_Truck_8,Asset_ID_Truck_9
0,-65.7383,11.2497,390,27.0,67.8,10.023475,38,320,4,60.1,285,1,0,2,0,1,0,0,0,0,0,0,0,1,0,0
1,22.2748,-131.7086,491,22.5,54.3,21.397599,16,439,7,80.9,174,1,7,2,0,0,1,0,0,0,0,0,1,0,0,0
2,54.9232,79.5455,190,25.2,62.2,14.520535,34,355,3,99.2,260,0,18,0,1,1,0,1,0,0,0,0,0,0,0,0
3,42.3900,-1.4788,330,25.4,52.3,2.613761,37,227,5,97.4,160,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,-65.8477,47.9468,480,20.5,57.2,35.412586,56,197,6,71.6,270,1,15,4,0,0,0,0,0,0,0,0,0,1,0,0
